In [1]:
import geopandas as gpd
from shapely.geometry import Point

# Cesty k souborům (přizpůsob si je podle potřeby)
linie_path = "czech_linie_trimmed.gpkg"
centroidy_path = "../data/czech/czech_regions_points.gpkg"
output_path = "czech_linie_oriented.gpkg"

# Načtení dat
gdf_lines = gpd.read_file(linie_path)
gdf_points = gpd.read_file(centroidy_path)

# Ujisti se, že obě vrstvy mají stejné souřadnicové systémy
gdf_points = gdf_points.to_crs(gdf_lines.crs)

# Výpočet orientace linie:
#  1 = linie začíná v centroidu
# -1 = linie končí v centroidu
#  0 = ani začátek ani konec není v blízkosti centroidu (výjimečný případ)

def determine_orientation(row, centroids, tolerance=100):
    start = row.geometry.coords[0]
    end = row.geometry.coords[-1]
    start_point = Point(start)
    end_point = Point(end)

    if centroids.distance(start_point).min() < tolerance:
        return 1
    elif centroids.distance(end_point).min() < tolerance:
        return -1
    else:
        return 0

# Použij funkci a vytvoř nový sloupec
gdf_lines["orientation"] = gdf_lines.apply(
    determine_orientation,
    axis=1,
    centroids=gdf_points.geometry
)

# Export do nového souboru GeoPackage
gdf_lines.to_file(output_path, driver="GPKG", layer="linie_oriented")

print("✅ Hotovo! Výstup uložen do:", output_path)


✅ Hotovo! Výstup uložen do: czech_linie_oriented.gpkg
